# **Option PnL Explained using Finite Difference and MultiVariate Taylor's Series**

In [ ]:
import numpy as np
import scipy.stats as si

### Fair value of  European Vanilla Call with Dividend 

In [ ]:
def European_call_val(St, K, T, t, r, vol):
  
  d1 = (np.log(St / K) + (r + 0.5 * vol*vol) * (T-t)) / (vol * np.sqrt((T-t)))
  d2 = (np.log(St / K) + (r - 0.5 * vol*vol) * (T-t)) / (vol * np.sqrt((T-t)))
    
  fair_val = (St * si.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * (T-t)) * si.norm.cdf(d2, 0.0, 1.0))
    
  return fair_val

In [ ]:
European_call_val(100,100,0.25,0,0.05,0.30)

6.583084497992466

###Option Greeks using Finite Difference and Closed Form Solution (a.k.a Analytical Solution)

#### **1. Using Finite Difference**

Note: Shock size on stock price is taken as $0.01 and shock size on vol is taken as 0.1% absolute

In [ ]:
#Only forward delta is shown. Backward and Central can be easily implemented based on the below code. FD stands for Finite Difference
def European_call_delta_FD(shock_size, St, K, T, t, r, vol):
  
  base_price = European_call_val(St, K, T, t, r, vol)
  shocked_price_up = European_call_val(St+shock_size, K, T, t, r, vol)
  
  delta = (shocked_price_up - base_price)/(shock_size)
  
  return delta

In [ ]:
European_call_delta_FD(0.01,100,100,0.25,0,0.05,0.30)

0.5630342436781177

In [ ]:
def European_call_gamma_FD(shock_size, St, K, T, t, r, vol):
  
  base_price = European_call_val(St, K, T, t, r, vol)
  shocked_price_up = European_call_val(St+shock_size, K, T, t, r, vol)
  shocked_price_down = European_call_val(St-shock_size, K, T, t, r, vol)
  
  gamma = (shocked_price_up + shocked_price_down - 2* base_price)/(shock_size*shock_size)
  
  return gamma

In [ ]:
European_call_gamma_FD(0.01,100,100,0.25,0,0.05,0.30)

0.02626485667178713

In [ ]:
def European_call_vega_FD(shock_size, St, K, T, t, r, vol):
  
  base_price = European_call_val(St, K, T, t, r, vol)
  shocked_price_up = European_call_val(St, K, T, t, r, vol+shock_size)

  vega = (shocked_price_up - base_price)/(shock_size)
  
  return vega

In [ ]:
European_call_vega_FD(0.001,100,100,0.25,0,0.05,0.30)

19.698685353795042

In [ ]:
def European_call_vomma_FD(shock_size, St, K, T, t, r, vol):
  
  base_price = European_call_val(St, K, T, t, r, vol)
  shocked_price_up = European_call_val(St, K, T, t, r, vol+shock_size)
  shocked_price_down = European_call_val(St, K, T, t, r, vol-shock_size)

  vomma = (shocked_price_up + shocked_price_down - 2* base_price)/(shock_size*shock_size)
  
  return vomma

In [ ]:
European_call_vomma_FD(0.001,100,100,0.25,0,0.05,0.30)

0.08664261486046598

In [ ]:
def European_call_vanna_FD(shock_size_stock, shock_size_vol, St, K, T, t, r, vol):
  
  base_price = European_call_val(St, K, T, t, r, vol)
  shocked_price_upstock_upvol = European_call_val(St+shock_size_stock, K, T, t, r, vol+shock_size_vol)
  shocked_price_upstock_downvol = European_call_val(St+shock_size_stock, K, T, t, r, vol-shock_size_vol)
  shocked_price_downstock_upvol = European_call_val(St-shock_size_stock, K, T, t, r, vol+shock_size_vol)
  shocked_price_downstock_downvol = European_call_val(St-shock_size_stock, K, T, t, r, vol-shock_size_vol)
  
  vanna = (shocked_price_upstock_upvol - shocked_price_upstock_downvol - shocked_price_downstock_upvol + shocked_price_downstock_downvol)/(4*shock_size_stock*shock_size_vol)
  
  return vanna

In [ ]:
European_call_vanna_FD(0.01,0.001,100,100,0.25,0,0.05,0.30)

-0.010944858530592681

#### **2. Closed Form**
**Note: This can be better implemented using a class.**
Same inputs are going to each function. d1 and d2 calculated multiple times, vega recalculated in vomma, etc are all inefficiencies that can be addressed by creating a Closed_Form_Greeks_Class. This will be covered in subsequent lectures. Class is not included here to make the understanding of the topic simple

In [ ]:
#CF stands for Closed Form

def European_call_delta_CF(St, K, T, t, r, vol):
  
  d1 = (np.log(St / K) + (r + 0.5 * vol*vol) * (T-t)) / (vol * np.sqrt((T-t)))
    
  delta = si.norm.cdf(d1, 0.0, 1.0)

  return delta

In [ ]:
opt_delta = European_call_delta_CF(100,100,0.25,0,0.05,0.30)
print(opt_delta)

0.5629029283920401


In [ ]:
def European_call_gamma_CF(St, K, T, t, r, vol):
  
  d1 = (np.log(St / K) + (r + 0.5 * vol*vol) * (T-t)) / (vol * np.sqrt((T-t)))
    
  gamma = (si.norm.pdf(d1, 0.0, 1.0)) / (St * vol * np.sqrt(T-t))

  return gamma

In [ ]:
opt_gamma = European_call_gamma_CF(100,100,0.25,0,0.05,0.30)
print(opt_gamma)

0.02626485733014476


In [ ]:
def European_call_vega_CF(St, K, T, t, r, vol):
  
  d1 = (np.log(St / K) + (r + 0.5 * vol*vol) * (T-t)) / (vol * np.sqrt((T-t)))
    
  vega = St * si.norm.pdf(d1, 0.0, 1.0) * np.sqrt(T-t) 

  return vega

In [ ]:
opt_vega = European_call_vega_CF(100,100,0.25,0,0.05,0.30)
print(opt_vega)

19.69864299760857


In [ ]:
def European_call_vomma_CF(St, K, T, t, r, vol):
  
  d1 = (np.log(St / K) + (r + 0.5 * vol*vol) * (T-t)) / (vol * np.sqrt((T-t)))
  d2 = d1 - vol*np.sqrt(T-t)  
  vega = St*si.norm.pdf(d1, 0.0, 1.0)*np.sqrt(T-t)
  
  vomma = vega*d1*d2/vol

  return vomma

In [ ]:
opt_vomma = European_call_vomma_CF(100,100,0.25,0,0.05,0.30)
print(opt_vomma)

0.08663755022096391


In [ ]:
def European_call_vanna_CF(St, K, T, t, r, vol):
  
  d1 = (np.log(St / K) + (r + 0.5 * vol*vol) * (T-t)) / (vol * np.sqrt((T-t)))
  d2 = d1 - vol*np.sqrt(T-t)
    
  vanna = -si.norm.pdf(d1, 0.0, 1.0)*d2/vol

  return vanna

In [ ]:
opt_vanna = European_call_vanna_CF(100,100,0.25,0,0.05,0.30)
print(opt_vanna)

-0.010943690554227017


# Explained PnL Attribution using MultiVariate Taylor's Series
The unexplained PnL is captured via risk not in VaR through stress testing

In [ ]:
chg_stock_price = 0.05
chg_vol = 0.004

#This is the exposure that you need to hedge in order to hedge the greek. 
#Typically, in equities, traders hedge stock price and volatility based greeks and not theta and rho
Delta_PnL = opt_delta * chg_stock_price
Gamma_PnL = 0.5 * opt_gamma * chg_stock_price * chg_stock_price
Vega_PnL = opt_vega * chg_vol
Vomma_PnL = 0.5 * opt_vomma * chg_vol * chg_vol
Vanna_PnL = opt_vanna * chg_stock_price * chg_vol

Total_Explained_PnL = Delta_PnL +  Gamma_PnL + Vega_PnL + Vomma_PnL + Vanna_PnL

print("Delta PnL: ", Delta_PnL)
print("Gamma PnL: ", Gamma_PnL)
print("Vega PnL: ", Vega_PnL)
print("Vomma PnL: ", Vomma_PnL)
print("Vanna PnL: ", Vanna_PnL)
print("Total Explained PnL: ", Total_Explained_PnL)
print("\n")

#Attribution Analysis
print("Delta PnL Attribution: ", Delta_PnL/Total_Explained_PnL)
print("Gamma PnL Attribution: ", Gamma_PnL/Total_Explained_PnL)
print("Vega PnL Attribution: ", Vega_PnL/Total_Explained_PnL)
print("Vomma PnL Attribution: ", Vomma_PnL/Total_Explained_PnL)
print("Vanna PnL Attribution: ", Vanna_PnL/Total_Explained_PnL)

Delta PnL:  0.02814514641960201
Gamma PnL:  3.2831071662680955e-05
Vega PnL:  0.07879457199043428
Vomma PnL:  6.931004017677113e-07
Vanna PnL:  -2.1887381108454035e-06
Total Explained PnL:  0.10697105384398989


Delta PnL Attribution:  0.2631099293520079
Gamma PnL Attribution:  0.000306915473699669
Vega PnL Attribution:  0.7365971368792055
Vomma PnL Attribution:  6.479326666993033e-06
Vanna PnL Attribution:  -2.0461031579977995e-05


# PnL Explained using Analytical solution (Full Reval)

In [ ]:
Total_PnL = European_call_val((100 + chg_stock_price),100,0.25,0,0.05,(0.30 + chg_vol)) - European_call_val(100,100,0.25,0,0.05,0.30)
print("Total PnL:", Total_PnL)
print("Total Explained PnL:", Total_Explained_PnL)
print("Total Unexplained PnL:", Total_PnL - Total_Explained_PnL)

Total PnL: 0.10697083815418296
Total Explained PnL: 0.10697105384398989
Total Unexplained PnL: -2.1568980693165152e-07
